In [0]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_unixtime, to_date, col, lower, regexp_replace, dense_rank, when
from pyspark.conf import SparkConf
from pyspark.sql.window import Window
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
# Cargamos los datos que guardamos del proceso etl de entrenamiento
train_df = spark.read.format("parquet").load("/mnt/backupaws/Datos_ML_BAR/Datos_Bar_entrenamiento/part-00000-tid-7141405282916243732-b17bb745-8483-4133-93dc-d23f740f7603-2214-1-c000.snappy.parquet")

In [0]:
# Cargamos los datos que guardamos del proceso etl de entrenamiento
test_df = spark.read.format("parquet").load("/mnt/backupaws/Datos_ML_BAR/Datos_Bar_testeo/part-00000-tid-6453973100730320895-f76a13df-0423-4b94-a042-3de55daa595d-2231-1-c000.snappy.parquet")

In [0]:
# Llamamos al modelo e indicamos las columnas a usar para el modelo de recomendacion
als = ALS(userCol="id_name", itemCol="id_name_empresa", ratingCol="rating", coldStartStrategy="drop")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-239659893837730>:2
      1 # Llamamos al modelo e indicamos las columnas a usar para el modelo de recomendacion
----> 2 als = ALS(userCol="id_name", itemCol="id_name_empresa", ratingCol="rating", coldStartStrategy="drop")

NameError: name 'ALS' is not defined

In [0]:
# Colocamos los parametros a usar para el cross validator
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 50, 100]) \
    .addGrid(als.regParam, [0.01, 0.05, 0.1]) \
    .addGrid(als.alpha, [1, 5, 10]) \
    .addGrid(als.nonnegative, [True]) \
    .addGrid(als.implicitPrefs, [True, False]) \
    .build()

In [0]:
# Definir evaluador
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [0]:
# Definir objeto CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [0]:
# Ajustar modelo y buscar mejores parámetros
cv_model = cv.fit(train_df)

In [0]:
# Evaluar modelo en conjunto de pruebas
predictions = cv_model.transform(test_df)
rmse = evaluator.evaluate(predictions)
print("RMSE en conjunto de pruebas: {:.4f}".format(rmse))

In [0]:
# Supongamos que `model` es el modelo que quieres guardar
model_path = "/mnt/backupaws/metadata/modelo_mejorado_bar"  # Ruta donde se guardará el modelo en el sistema de archivos de Azure
cv_model.write().overwrite().save(model_path)

In [0]:
# Obtener los parámetros del mejor modelo
bestModel = cv_model.bestModel
params = bestModel.extractParamMap()
print("Los parámetros del mejor modelo son:")
for key, value in params.items():
    print("{}: {}".format(key.name, value))

In [0]:
best_model = cv_model.bestModel

In [0]:
from pyspark.sql.functions import col, expr, desc
from pyspark.ml.recommendation import ALSModel

# Predicciones para cada usuario
userRecs = best_model.recommendForAllUsers(100)

# Selección de las 10 películas más recomendadas para cada usuario
userRecs = userRecs.select(
    col("id_name"),
    expr("explode(recommendations)").alias("recommendation")
).select(
    col("id_name"),
    col("recommendation.id_name_empresa"),
    col("recommendation.rating")
)

# Ordenamiento de las películas según su predicción
userRecs = userRecs.orderBy(
    col("id_name"), desc("rating")
)

In [0]:
# Mostrar las 10 restaurantes más recomendadas para cada usuario
userRecs.show(10)

In [0]:
userRecs.write.mode("overwrite").parquet("/mnt/backupaws/metadata_bar/Users_recomendations")